In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
url = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
response = requests.get(url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")

Success


In [3]:
import re
results_page = BeautifulSoup(response.content,'lxml')

#### get data from January 2009 through June 2015

In [13]:
def get_taxi_link():
    link_array = []
    pattern = r"(http.*yellow_tripdata_(20[0|1][0-9])-(0?\d*).parquet)"
    # pattern = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet"
    match = re.findall(pattern, str(results_page))
    for i in match:
        if int(i[1]) < 2015:
            link_array.append(i[0])
        elif int(i[1]) == 2015 and int(i[2]) <= 6:
            link_array.append(i[0])
    return link_array

In [14]:
link = get_taxi_link()
link

['https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2015-06.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-01.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-02.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-03.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-04.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-05.parquet',
 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2014-06.pa

In [6]:
pattern = r"yellow.*.parquet"
match = re.search(pattern, link[0])
match.group()

'yellow_tripdata_2019-01.parquet'

In [21]:
!True

In [24]:
import os

if not os.path.exists("yellow_tripdata_2009-01.parquet"):
    print("true")
else:
    print("doesn't exist")

doesn't exist


In [25]:
import os

pattern = r"yellow.*.parquet"
for taxi_data_url in link:
    match = re.search(pattern, taxi_data_url)
    name_of_file = match.group()
    
    # to make sure we're not re-downloading the same file again
    if not os.path.exists(name_of_file):
        response = requests.get(taxi_data_url, stream=True)
        with open(name_of_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                

In [8]:
link[0]

'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2019-01.parquet'

In [9]:
import pandas as pd
from pyarrow.parquet import ParquetFile
df = pd.read_parquet(link[0])

In [10]:
index = df.columns

In [11]:
np.array(index)

array(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID',
       'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount',
       'congestion_surcharge', 'airport_fee'], dtype=object)

In [12]:
import geopandas as gpd
shapefile = gpd.read_file("taxi_zones.shp")
shapefile

DriverError: taxi_zones.shp: No such file or directory

According to the shapefile "taxi_zones.shp" above, the highest objectID, which is the pickup location ID and drop off location ID, is 263. Therefore, in the Cleaning & filtering step, we remove the IDs that are greater than 263.

In [ ]:
nc_counties = shapefile['geometry']
nc_counties = nc_counties.to_crs(4326)
nc_counties['lon'] = nc_counties.centroid.x
nc_counties['lat'] = nc_counties.centroid.y

In [ ]:
nc_counties.y

In [ ]:
nc_counties['lon']